In [1]:
import numpy as np
import pandas as pd
from SARLens.processor.focus import coarseRDA
from SARLens.utils.io import load, plot_with_cdf

Zarr packet not installed. Zarr functionalities will not work.


# Metadata checker:

In [1]:
from SARLens.processor.decode import header_extractor

In [2]:
inputfile = '/Users/robertodelprete/Desktop/AutoFocusNet/Data/DATA/S1A_S3_RAW__0SDH_20240524T213606_20240524T213631_054018_069139_241A.SAFE/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139.dat'
meta_rich = header_extractor(inputfile, 'richa')
meta_s1isp = header_extractor(inputfile, 's1isp')

decoded: 48941 packets [00:01, 32234.65 packets/s]


In [58]:
meta_rich.to_pickle('meta_rich.pkl')
meta_s1isp.to_pickle('meta_s1isp.pkl')

In [8]:
richa_to_s1isp = {
    "Packet Version Number": "packet_version_number",
    "Packet Type": "packet_type",
    "Secondary Header Flag": "secondary_header_flag",
    "PID": "pid",
    "PCAT": "pcat",
    "Sequence Flags": "sequence_flags",
    "Packet Sequence Count": "packet_sequence_count",
    "Packet Data Length": "packet_data_length",
    "Coarse Time": "coarse_time",
    "Fine Time": "fine_time",
    "Sync": "sync_marker",
    "Data Take ID": "data_take_id",
    "ECC Number": "ecc_num",
    "Test Mode": "test_mode",
    "Rx Channel ID": "rx_channel_id",
    "Instrument Configuration ID": "instrument_configuration_id",
    "Sub-commutated Ancilliary Data Word Index": "data_word_index",
    "Space Packet Count": "space_packet_count",
    "PRI Count": "pri_count",
    "Error Flag": "error_flag",
    "BAQ Mode": "baq_mode",
    "BAQ Block Length": "baq_block_length",
    "Range Decimation": "range_decimation",
    "Rx Gain": "rx_gain",
    "Tx Ramp Rate": "tx_ramp_rate",
    "Tx Pulse Start Frequency": "tx_pulse_start_freq",
    "Tx Pulse Length": "tx_pulse_length",
    "Rank": "rank",
    "PRI": "pri",
    "SWST": "swst",
    "SWL": "swl",
    "SAS SSB Flag": "ssb_flag",
    "Polarisation": "polarization",
    "Temperature Compensation": "temperature_compensation",
    "Calibration Mode": "cal_mode",
    "Tx Pulse Number": "tx_pulse_number",
    "Signal Type": "signal_type",
    "Swap Flag": "swap",
    "Swath Number": "swath_number",
    "Number of Quads": "number_of_quads"
}


In [60]:
VERBOSE = False

idx = 129

row_rich = meta_rich.iloc[idx]
row_s1isp = meta_s1isp.iloc[idx]

keys = list(row_rich.keys())

non_equal_elements = []

for key in keys:
    try: 
        elem_rich = row_rich[key]
        elem_s1isp = row_s1isp[richa_to_s1isp[key]]
        
        if VERBOSE:
            print(key, elem_rich, float(elem_s1isp), elem_rich == float(elem_s1isp))
        
        if not elem_rich == float(elem_s1isp):
            non_equal_elements.append((key, float(elem_rich), float(elem_s1isp)))

    except:
        print('Missing key:', key)
        continue

for elem in non_equal_elements:
    print(elem)

Missing key: Sub-commutated Ancilliary Data Word
('Packet Sequence Count', 1.0, 129.0)
('Packet Data Length', 7654.0, 7653.0)
('Fine Time', 0.35900115966796875, 23527.0)
('Rx Gain', -6.0, 12.0)
('Tx Ramp Rate', 1344932774550.9956, 34770.0)
('Tx Pulse Start Frequency', -29704503.224123616, 12970.0)
('Tx Pulse Length', 4.41724329115483e-05, 1658.0)
('PRI', 0.0005194923216780943, 19499.0)
('SWST', 9.593783529221075e-05, 3601.0)
('SWL', 4.683663272527256e-05, 1758.0)


## Focusing:

In [ ]:
radar_data = load('/Users/robertodelprete/Desktop/AutoFocusNet/Data/tmp/DATA/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139_pkt_0.pkl')
metadata = load('/Users/robertodelprete/Desktop/AutoFocusNet/Data/tmp/DATA/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139_pkt_0_metadata.pkl')
ephemeris = load('/Users/robertodelprete/Desktop/AutoFocusNet/Data/tmp/DATA/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139_ephemeris.pkl')

In [ ]:
# plot_with_cdf(radar_data, figsize=(6,6))

In [ ]:
RadarProcessor = coarseRDA(
    raw_data={'echo':radar_data[:,:], 'ephemeris':ephemeris, 'metadata':metadata}, 
    verbose=False, 
    backend='custom'
)


In [ ]:
def multiply(a, b):
    """
    Multiplies two arrays a and b using the specified backend.
    """
    return a * b

In [ ]:
RadarProcessor.fft2D()

# RG compression
RadarProcessor.radar_data = multiply(RadarProcessor.radar_data, 
                                RadarProcessor.get_range_filter())

# RCMC
RadarProcessor.radar_data = multiply(RadarProcessor.radar_data, 
                                RadarProcessor.get_RCMC())

# IFFT Range #TODO: address this in a backend manner for supporting torch
RadarProcessor.ifft_rg()

# Az Compression
RadarProcessor.radar_data = multiply(RadarProcessor.radar_data, 
                            RadarProcessor.get_azimuth_filter())


# IFFT #TODO: address this in a backend manner for supporting torch
RadarProcessor.ifft_az()